In [1]:
import pandas as pd
import numpy as np
import S2_0_Loading_Data
from keras.applications import vgg16
from tensorflow.keras.applications import InceptionResNetV2, EfficientNetB7
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.regularizers import L2
from keras.callbacks import EarlyStopping
import keras
import S4_0_Helper_Functions

In [2]:
train_x, test_x, train_y, test_y = S2_0_Loading_Data.load_data(image_size=(100, 100))
train_x.shape

(3257, 100, 100, 3)

In [3]:
pretrained_vgg = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
pretrained_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [4]:
weight_decay = 0.0005
for layer in pretrained_vgg.layers:
    layer.trainable = False
model = Sequential()
model.add(pretrained_vgg)
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=L2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 512)               2359808   
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [5]:
# compile model
model.compile(
    optimizer='rmsprop',
    loss='CategoricalCrossentropy',
    metrics=['accuracy']
)


In [6]:
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 10,
    verbose = 1,
    callbacks = [callback]
)

Epoch 1/10
102/102 [==============================] - 78s 763ms/step - loss: 1.0518 - accuracy: 0.8081 - val_loss: 1.0099 - val_accuracy: 0.7963
Epoch 2/10
102/102 [==============================] - 78s 767ms/step - loss: 0.4592 - accuracy: 0.9364 - val_loss: 1.4329 - val_accuracy: 0.8491
Epoch 3/10
102/102 [==============================] - 79s 777ms/step - loss: 0.3378 - accuracy: 0.9564 - val_loss: 1.0742 - val_accuracy: 0.8748
Epoch 4/10
102/102 [==============================] - 76s 750ms/step - loss: 0.2777 - accuracy: 0.9628 - val_loss: 1.4771 - val_accuracy: 0.8184
Epoch 5/10
102/102 [==============================] - 78s 770ms/step - loss: 0.2403 - accuracy: 0.9705 - val_loss: 0.6223 - val_accuracy: 0.9104
Epoch 6/10
102/102 [==============================] - 79s 778ms/step - loss: 0.2271 - accuracy: 0.9690 - val_loss: 0.6579 - val_accuracy: 0.9153
Epoch 7/10
102/102 [==============================] - 78s 766ms/step - loss: 0.2236 - accuracy: 0.9699 - val_loss: 0.8594 - val_ac

Accuracy peaks at around 92%. We suspect that with a better computing and more time, having more epochs may increase accuracy further.

In [7]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.7705521472392638
recall: 0.7705521472392638
precision: 0.7705521472392638


In [8]:
keras.backend.clear_session()

In [9]:
pretrained_ENB7 = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

In [10]:
weight_decay = 0.0005
for layer in pretrained_ENB7.layers:
    layer.trainable = False
model = Sequential()
model.add(pretrained_ENB7)
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=L2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [11]:
# compile model
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='CategoricalCrossentropy',
              metrics=METRICS)

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional)  (None, 4, 4, 2560)       64097687  
                                                                 
 flatten (Flatten)           (None, 40960)             0         
                                                                 
 dense (Dense)               (None, 512)               20972032  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [12]:
callback = EarlyStopping(monitor='val_prc', mode='max',min_delta = 0.01, verbose=1, patience=10,restore_best_weights=True)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 1, # increase if you have a good computer
    verbose = 1,
    callbacks = [callback]
)

102/102 [==============================] - 173s 2s/step - loss: 1.5060 - tp: 2303.0000 - fp: 683.0000 - tn: 5831.0000 - fn: 954.0000 - accuracy: 0.7430 - precision: 0.7713 - recall: 0.7071 - auc: 0.8720 - prc: 0.7732 - val_loss: 1.3300 - val_tp: 597.0000 - val_fp: 181.0000 - val_tn: 1449.0000 - val_fn: 218.0000 - val_accuracy: 0.7583 - val_precision: 0.7674 - val_recall: 0.7325 - val_auc: 0.8920 - val_prc: 0.8141


In [13]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.758282208588957
recall: 0.758282208588957
precision: 0.758282208588957


In [14]:
keras.backend.clear_session()

In [15]:
pretrained_IRNV2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

In [16]:
weight_decay = 0.0005
for layer in pretrained_IRNV2.layers:
    layer.trainable = False
model = Sequential()
model.add(pretrained_IRNV2)
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=L2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='CategoricalCrossentropy',
              metrics=METRICS)

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 activation_203 (Activation)  (None, 512)              0         
                                                                 
 batch_normalization_203 (Ba  (None, 512)              2048      
 tchNormalization)                                               
                                                                 
 dropout (Dropout)           (None, 512)               0

In [17]:
callback = EarlyStopping(monitor='val_prc', mode='max',min_delta = 0.01, verbose=1, patience=10,restore_best_weights=True)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 1, # increase if you have a good computer
    verbose = 1,
    callbacks = [callback]
)

102/102 [==============================] - 103s 889ms/step - loss: 0.9796 - tp: 2715.0000 - fp: 844.0000 - tn: 7300.0000 - fn: 1357.0000 - accuracy: 0.7274 - precision: 0.7629 - recall: 0.6667 - auc: 0.8679 - prc: 0.7794 - val_loss: 1.1042 - val_tp: 630.0000 - val_fp: 179.0000 - val_tn: 1451.0000 - val_fn: 185.0000 - val_accuracy: 0.7755 - val_precision: 0.7787 - val_recall: 0.7730 - val_auc: 0.9187 - val_prc: 0.8608


In [18]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.7754601226993865
recall: 0.7754601226993865
precision: 0.7754601226993865
